# Using the NYC Open Data API for GIS

This notebook explains how to query data from the NYC Open Data API, and integrate it into a GIS workflow. We'll use a simple example, pulling some data from [311 complaint records](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data) to make a map of rat sightings in New York City.

By the end of this tutorial, you should be able to:
1) Configure and manage a Python environment within ArcGIS Pro.
2) Interpret data, query, and API syntax from NYC Open Data.
3) Import and map data from the NYC Open Data API.

## Getting started
To replicate, download this file, open it in a Python notebook editor such as Google Colab, and then run the workflow that follows.

In [1]:
# install libraries that are not native to the AGOL environment
! pip install geopandas
! pip install sodapy

In [2]:
# load other necessary libraries
from google.colab import drive
from sodapy import Socrata
import folium
import os
import pandas
import geopandas

In [3]:
# and set up your working environment
# mount Google Drive
drive.mount('/content/drive')

default_folder = '/content/drive/MyDrive/Teaching/INFO 612 Advanced GIS' ## replace with yours
print("Directory:" + default_folder)

Mounted at /content/drive
Directory:/content/drive/MyDrive/Teaching/INFO 612 Advanced GIS


## Understanding the NYC Open Data API

Like many large municipalities in the US, [NYC Open Data](https://opendata.cityofnewyork.us/) manages its public-facing databases with a content management system called [Socrata](https://dev.socrata.com/). The Socrata platform allows one to query data both through download tools and through an API.

To access the API, navigate to the dataset you want and configure a query for the subset of data you want. Then instead of downloading a file choose the option for API endpoint.

Copy the URL, and then note how the query is written into the structure of the URL. After the main URL, the first part of the query defines the dataset and file format:
`erm2-nwe9`. After that the query you created will follow. In my case, I selected complaints about rodents in the 2025 calendar year.

In [20]:
# now load this basic data
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=SELECT%0A%20%20%60unique_key%60%2C%0A%20%20%60created_date%60%2C%0A%20%20%60closed_date%60%2C%0A%20%20%60agency%60%2C%0A%20%20%60agency_name%60%2C%0A%20%20%60complaint_type%60%2C%0A%20%20%60descriptor%60%2C%0A%20%20%60location_type%60%2C%0A%20%20%60incident_zip%60%2C%0A%20%20%60incident_address%60%2C%0A%20%20%60street_name%60%2C%0A%20%20%60cross_street_1%60%2C%0A%20%20%60cross_street_2%60%2C%0A%20%20%60intersection_street_1%60%2C%0A%20%20%60intersection_street_2%60%2C%0A%20%20%60address_type%60%2C%0A%20%20%60city%60%2C%0A%20%20%60landmark%60%2C%0A%20%20%60facility_type%60%2C%0A%20%20%60status%60%2C%0A%20%20%60due_date%60%2C%0A%20%20%60resolution_description%60%2C%0A%20%20%60resolution_action_updated_date%60%2C%0A%20%20%60community_board%60%2C%0A%20%20%60bbl%60%2C%0A%20%20%60borough%60%2C%0A%20%20%60x_coordinate_state_plane%60%2C%0A%20%20%60y_coordinate_state_plane%60%2C%0A%20%20%60open_data_channel_type%60%2C%0A%20%20%60park_facility_name%60%2C%0A%20%20%60park_borough%60%2C%0A%20%20%60vehicle_type%60%2C%0A%20%20%60taxi_company_borough%60%2C%0A%20%20%60taxi_pick_up_location%60%2C%0A%20%20%60bridge_highway_name%60%2C%0A%20%20%60bridge_highway_direction%60%2C%0A%20%20%60road_ramp%60%2C%0A%20%20%60bridge_highway_segment%60%2C%0A%20%20%60latitude%60%2C%0A%20%20%60longitude%60%2C%0A%20%20%60location%60%0AWHERE%0A%20%20%60created_date%60%0A%20%20%20%20BETWEEN%20%222025-01-01T11%3A12%3A18%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20AND%20%222025-08-04T11%3A12%3A18%22%20%3A%3A%20floating_timestamp%0A%20%20AND%20caseless_eq(%60complaint_type%60%2C%20%22Rodent%22)%0AORDER%20BY%20%60created_date%60%20DESC%20NULL%20FIRST"
test = pandas.read_csv(url)
test.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,65762468,2025-08-04T11:09:47.000,2025-08-04T12:30:57.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,3+ Family Apt. Building,11215,477 THIRD STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.670958,-73.979459,"\n, \n(40.67095759491754, -73.97945938432618)"
1,65753924,2025-08-04T11:00:19.000,2025-08-04T12:00:21.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,1-2 Family Dwelling,11369,30-11 86 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.761208,-73.882596,"\n, \n(40.76120770771694, -73.88259593041018)"
2,65753925,2025-08-04T10:43:36.000,2025-08-07T12:28:10.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,Other (Explain Below),10024,589 AMSTERDAM AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.789261,-73.973851,"\n, \n(40.789260886149705, -73.97385066924859)"
3,65756367,2025-08-04T10:42:47.000,2025-08-04T10:42:47.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,11207,411 BARBEY STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.672620,-73.887474,"\n, \n(40.67261988508562, -73.88747438960826)"
4,65762466,2025-08-04T10:41:02.000,2025-08-04T10:41:02.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,1-2 Family Dwelling,11369,23-52 94 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.766899,-73.876019,"\n, \n(40.76689890785611, -73.87601916588406)"


In [21]:
# convert to a spatial data frame
test_spatial = geopandas.GeoDataFrame(test,
                                      geometry=geopandas.points_from_xy(test.longitude, test.latitude),
                                      crs="EPSG:4326")

print(f"Dimensions: {test_spatial.shape}")

Dimensions: (1000, 42)


In [22]:
# create a map to view data
test_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# add the spatial data frame
folium.GeoJson(test_spatial).add_to(test_map)

test_map

## Design an API query

Note that the NYC Open Data API limits this kind of public API call to 1000 records. But most of the time you want more records than this (e.g. the query above has over 20,000 complaints). To get around this problem, we'll use more advanced functions of the API to access the data.

First off, you need to create an account with Tyler Technologies (the company that owns Socrata) and get a free API token. Here's [their guide](https://support.socrata.com/hc/en-us/articles/210138558-Generating-App-Tokens-and-API-Keys) on how to create app tokens and API keys.

Create an account at evergreen.data.socrata.com and then log in. Click your name near the top right corner and look for Developer Settings. Note I had trouble seeing this on some browsers, until I clicked on my name twice.

Create a new app token and then copy it for use in the script below. **Note:** Don't share your token or make it publicaly visible.

In [9]:
#set up for Socrata API
data_url = 'data.cityofnewyork.us'  # NYC Open Data is the platform we'll query
app_token = 'your key here' # Add your token here
client = Socrata(data_url, app_token, timeout=1000)  # configuring the Socrata client
print(client)

Querying the API requires an understanding of how the query is built into the URL. For our purposes, the query has two elements: a `complaint_type` about rodents and a time range for the `created_data`. We'll use a WHERE query to build that URL.

We first define the data set `erm2-nwe9`.

Then we create a query for time `created_date between '2025-01-01T00:00:00' and '2025-12-31T23:59:59'`, using `AND` to continue the query to include a complaint type `complaint_type like '%Rodent%'`. Note that I'm using a wildcard operator (%) to search for related terms.

Finally we'll specify a limit because the API call requires it. It can be as high as you want (in my case, there's only about 21,000 rows so a limit of 50000 is just fine). But please be a reasonable professional and avoid pulling massive amounts of data that you don't actually need.

For building your own queries, note that NYC Open Data has an API guide for each dataset. The guide for 311 complaints is [located here](https://dev.socrata.com/foundry/data.cityofnewyork.us/erm2-nwe9).

In [10]:
# first define the data set
dataset_identifier = "erm2-nwe9"

# then the query
query = "created_date between '2025-01-01T00:00:00' and '2025-12-31T23:59:59' AND complaint_type like '%Rodent%'"

# and the limit
limit = 50000

print("Dataset: " + dataset_identifier)
print("Query: " + query)

Dataset: erm2-nwe9
Query: created_date between '2025-01-01T00:00:00' and '2025-12-31T23:59:59' AND complaint_type like '%Rodent%'


In [ ]:
# now submit the query
results = client.get(dataset_identifier,
                     where=query,
                     limit=limit)
print("Query successfully submitted.")

Query successfully submitted.


In [11]:
# Alternatively, you can write the query directly into the API call like this:
results = client.get("erm2-nwe9",
                     where="created_date between '2025-01-01T00:00:00' and '2025-12-31T23:59:59' AND complaint_type like '%Rodent%'",
                     order="created_date DESC",
                     limit=50000)
print("Query successfully submitted.")

Query successfully submitted.


In [23]:
# read data into data frame
data = pandas.DataFrame.from_records(results)
data.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,closed_date,facility_type,resolution_description,resolution_action_updated_date
0,66653342,2025-10-30T00:31:58.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Sidewalk,10003,219 EAST 11 STREET,EAST 11 STREET,...,ONLINE,Unspecified,MANHATTAN,40.73090444753835,-73.98759887572585,"{'latitude': '40.73090444753835', 'longitude':...",NaN,NaN,NaN,NaN
1,66653341,2025-10-29T23:58:08.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Mixed Use Building,11232,3906 6 AVENUE,6 AVENUE,...,PHONE,Unspecified,BROOKLYN,40.64983559018456,-74.00183070010809,"{'latitude': '40.64983559018456', 'longitude':...",NaN,NaN,NaN,NaN
2,66656657,2025-10-29T23:27:24.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Mixed Use Building,10021,490 EAST 74 STREET,EAST 74 STREET,...,ONLINE,Unspecified,MANHATTAN,40.76835869687328,-73.95325933716485,"{'latitude': '40.76835869687328', 'longitude':...",NaN,NaN,NaN,NaN
3,66658883,2025-10-29T22:53:25.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,10029,310 EAST 112 STREET,EAST 112 STREET,...,ONLINE,Unspecified,MANHATTAN,40.79430639282482,-73.93927159922073,"{'latitude': '40.79430639282482', 'longitude':...",NaN,NaN,NaN,NaN
4,66653340,2025-10-29T22:41:09.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Mixed Use Building,11106,31-50 33 STREET,33 STREET,...,ONLINE,Unspecified,QUEENS,40.762386261386,-73.92285792230227,"{'latitude': '40.762386261386', 'longitude': '...",NaN,NaN,NaN,NaN


In [24]:
# drop nulls
data = data.dropna(subset=['latitude', 'longitude'])

# convert to a spatial data frame
data_spatial = geopandas.GeoDataFrame(data,
                                      geometry=geopandas.points_from_xy(data.longitude, data.latitude),
                                      crs="EPSG:4326")
print(f"Dimensions: {data_spatial.shape}")

Dimensions: (28663, 34)


In [18]:
# to save as a geojson
save_path = os.path.join(default_folder, "socrata_data.geojson")
data_spatial.to_file(save_path, driver='GeoJSON')
print(f"Saved geojson to {save_path}")

Saved geojson to /content/drive/MyDrive/Teaching/INFO 612 Advanced GIS/socrata_data.geojson


## Futher reading
- Bauer, Mark, "Socrata Open Data API Tutorial with Python and NYC Open Data", https://github.com/mebauer/sodapy-tutorial-nyc-opendata
- Socrata "SODA Developers Guide", https://dev.socrata.com/
- Tyler Technologies, "Generating app tokens and API keys", https://support.socrata.com/hc/en-us/articles/210138558-Generating-App-Tokens-and-API-Keys